In [1]:
# Set the project path to root
import os
os.chdir("C:/Users/Ryan/Documents/GitHub/transport-for-london")
from pathlib import Path
import pandas as pd
import random
import json
from src.data import api_tools


First up we'll explore the locations of Transport for London (TFL) bike stations around the city.

We will:
1. Initiate an API connection to the TfL Rest API
2. Extract locations from the BikePoint endpoint
3. Save the extracted data (Which is returned as a JSON) as a CSV

In [2]:
endpoint = "BikePoint"
filepath = "./csv/"
filename = "tfl_response.csv"

# We have built a custom API class to handle requests
tfl_api = api_tools.TfLApiClient()
response = tfl_api.make_request('BikePoint')

# We convert the returned JSON to a pandas dataframe and save
response_df = pd.DataFrame(response)
response_df.to_csv(filepath + filename, index=None)


2023-12-31 16:07:26 [INFO]: API request successful
2023-12-31 16:07:26 [INFO]: Returned 785 rows


Let's take a look at the returned data...

In [3]:
response_df.dtypes

$type                    object
id                       object
url                      object
commonName               object
placeType                object
additionalProperties     object
children                 object
childrenUrls             object
lat                     float64
lon                     float64
dtype: object

Here we can see there is supplementary information for each bike station held within the 'additionalProperties' column, which may be worth exploring further...

In [4]:
# Supplementary data on each bike station
print([prop['key'] for prop in response_df.loc[0]['additionalProperties']])
pd.DataFrame(response_df['additionalProperties'][0])[['key', 'value', 'modified']]

['TerminalName', 'Installed', 'Locked', 'InstallDate', 'RemovalDate', 'Temporary', 'NbBikes', 'NbEmptyDocks', 'NbDocks', 'NbStandardBikes', 'NbEBikes']


,key,value,modified
0,TerminalName,001023,2023-12-31T15:00:53.8Z
1,Installed,true,2023-12-31T15:00:53.8Z
2,Locked,false,2023-12-31T15:00:53.8Z
3,InstallDate,1278947280000,2023-12-31T15:00:53.8Z
4,RemovalDate,,2023-12-31T15:00:53.8Z
5,Temporary,false,2023-12-31T15:00:53.8Z
6,NbBikes,14,2023-12-31T15:00:53.8Z
7,NbEmptyDocks,5,2023-12-31T15:00:53.8Z
8,NbDocks,19,2023-12-31T15:00:53.8Z
9,NbStandardBikes,14,2023-12-31T15:00:53.8Z


'NbBikes', 'NbEmptyDocks', 'NbDocks' seem like they may be useful so we will include these columns in the top level data.

One way would be to run a lambda function over the whole df, extracting the data of interest but this can get messy. Instead we will simply create a new df from the desired columns, including the bike station id, or in other words we transform the data from its raw state to a useable state.



In [5]:

additional_cols = ['NbBikes', 'NbEmptyDocks', 'NbDocks']
new_data = list()

for item in response:
    new_dict = dict()

    #Copy through the top level details
    new_dict['id'] = item['id']
    new_dict['lat'] = item['lat']
    new_dict['lon'] = item['lon']

    # Iterate the additional properties to extract the desired values
    additional_props = item['additionalProperties']

    # We don't name the cols explicitly, allowing us to change our mind on the desired columns in the future
    for prop in additional_props:
        if prop['key'] in additional_cols:
            new_dict[prop['key']] = prop['value']

    new_data.append(new_dict)

# We'll call this our bike stations df to work with from here on
bike_stations_df = pd.DataFrame(new_data)

Interestingly there are stations claiming to have 0 docks in the data. We will remove these from the dataset as a bike station without any bike docks seems nonsensical, however these may be special cases worth looking into later.

In [6]:
# A little data cleaning and housekeeping
bike_stations_df[additional_cols] = bike_stations_df[additional_cols].astype(int)
bike_stations_df['id'] = bike_stations_df['id'].astype(str)
clean_df = bike_stations_df.query('NbDocks > 0').reset_index()

One more handy feature might be the ratio of bikes available vs the number of docks vacant at each station.  
We can run a simple lambda function over the dataframe to add this feature.

In [9]:
bike_stations_df['occupancy_ratio'] = bike_stations_df.apply(lambda x: round(x['NbBikes'] / x['NbDocks'],2), axis=1)

Now we have the Name, ID and locations of all available bike stations plus some useful properties.  
We can visualise these to better understand their distribution

In [10]:
import keplergl

from keplergl import KeplerGl
map_data = bike_stations_df[['id', 'lat', 'lon', 'occupancy_ratio'] + additional_cols]
tfl_bike_map = KeplerGl()
tfl_bike_map.add_data(data=map_data, name='tfl_bike_stations')


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [11]:
tfl_bike_map

KeplerGl(data={'tfl_bike_stations': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…

After tweaking a few parameters and doing some visual exploration, we can see the patterns of vacant vs full docks at each bike station. Depending on when the API is queried, data may reflect different patterns of users travelling to and from the central city and outer suburbs of London.

Some possible inferences:

1. There are less bikes available at inner city bike stations in the evening
2. The system requires some "slack", with more docking spaces available than bikes
3. There may be a daily pattern to bike station occupancy levels
4. The Southeast is underserved relative to other regions
